# Tic Tac Toe

## Project imports

In [6]:
import ast
import os
import sys

sys.path.append(os.path.abspath(".."))

import pandas as pd

## Classification training

### Loading playable game states

In [7]:
game_file = "game_state_targets"
path = "../data/"
ext = ".csv"
headers = ["state", "last_player", "target"]

games = pd.read_csv(path + game_file + ext)
games[headers[0]] = games[headers[0]].apply(ast.literal_eval)
print(games.head())

                            state  last_player  target
0  [0, -1, 1, 1, 0, -1, -1, 1, 1]            1       0
1   [1, 0, 1, 0, 0, 0, -1, 0, -1]           -1       1
2  [0, 0, 0, -1, 1, 1, -1, 1, -1]           -1       1
3   [1, 0, 0, -1, -1, 1, 0, 0, 0]           -1       2
4  [0, 1, 1, -1, -1, 0, 1, -1, 0]           -1       0


### Training

- Setup

In [8]:
import torch
import torch.nn as nn

from models.neural_net import TicTacToeNet

net = TicTacToeNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
num_epochs = 1000
batch_size = 100

states = torch.tensor(games["state"].to_list(), dtype=torch.float32)
targets = torch.tensor(games["target"].values, dtype=torch.long)

- Training

In [9]:
for epoch in range(num_epochs):
    permutation = torch.randperm(states.size()[0])
    states = states[permutation]
    targets = targets[permutation]

    for i in range(0, states.size()[0], batch_size):
        batch_states = states[i : i + batch_size]
        batch_targets = targets[i : i + batch_size]

        outputs = net(batch_states)
        loss: torch.Tensor = loss_fn(outputs, batch_targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

torch.save(net.state_dict(), path + "models/mlp_model.pth")

Epoch 1/1000, Loss: 2.1985342502593994
Epoch 1/1000, Loss: 2.188608169555664
Epoch 1/1000, Loss: 2.1818060874938965
Epoch 1/1000, Loss: 2.1757307052612305
Epoch 1/1000, Loss: 2.1454379558563232
Epoch 1/1000, Loss: 2.130679130554199
Epoch 1/1000, Loss: 2.1486024856567383
Epoch 1/1000, Loss: 2.1679623126983643
Epoch 1/1000, Loss: 2.1274759769439697
Epoch 1/1000, Loss: 2.1676127910614014
Epoch 1/1000, Loss: 2.166177988052368
Epoch 1/1000, Loss: 2.1140098571777344
Epoch 1/1000, Loss: 2.0803024768829346
Epoch 1/1000, Loss: 2.103760242462158
Epoch 1/1000, Loss: 2.1375153064727783
Epoch 1/1000, Loss: 2.056727886199951
Epoch 1/1000, Loss: 2.0602502822875977
Epoch 1/1000, Loss: 2.103616237640381
Epoch 1/1000, Loss: 2.0754144191741943
Epoch 1/1000, Loss: 2.086153507232666
Epoch 1/1000, Loss: 2.050755262374878
Epoch 1/1000, Loss: 2.049694061279297
Epoch 1/1000, Loss: 2.028301477432251
Epoch 1/1000, Loss: 2.0804171562194824
Epoch 1/1000, Loss: 2.0257201194763184
Epoch 1/1000, Loss: 1.9893592596054

### Testing

In [15]:
states = torch.tensor(games["state"].to_list(), dtype=torch.float32)
targets = torch.tensor(games["target"].values, dtype=torch.long)
net.eval()
outputs = net(states)

_, predicted = torch.max(outputs, 1)

accuracy = (predicted == targets).sum().item() / targets.size(0)
print(f"Test Accuracy: {round(accuracy * 100, 2)}%")

Test Accuracy: 92.35%


### Saving results to csv

In [16]:
games["mlp_results"] = predicted.numpy()
games.to_csv(path + "neural_net_results" + ext, index=False)